In [348]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np


In [335]:
def scrape_years_list(): 
    url = 'http://www.ironman.com/triathlon/events/americas/ironman/world-championship/results.aspx?p=1&rd=20171014&ps=20#axzz5YeQSboHx'
    r = requests.get(url).content
    soup = BeautifulSoup(r, 'html.parser')
    listy = soup.find_all('ul', id="raceResults")[0].find_all('a')
    dates = []
    years = []
    for a in listy: 
        year = re.findall(r'\d+', a['href'])
        dates.append(int(year[0]))
        years.append(int(year[0][:4]))
    print(dates)
    print(years)
    return dates, years 

In [336]:
dates, years = scrape_years_list()


[20181013, 20171014, 20161008, 20151010, 20141011, 20131012, 20121013, 20111008, 20101009, 20091010, 20081011, 20071013, 20061021, 20051015, 20041016, 20031018, 20021019]
[2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002]


In [349]:
def scrape_categories(): 
    listy2 = soup.find_all('select', id="categorySelect")[0].find_all('option')
    agegroups = []
    for option in listy2: 
        cat = option['value']
        agegroups.append(cat)
    categories = agegroups[1:12]
    print('categores:',categories)
    return categories

In [350]:
categories = scrape_categories()

categores: ['Pro', '18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69']


In [351]:
def scraper_category(date, year):
    date = date
    year = year
    Raw = []
    for cat in categories: 
        if cat != '90 Plus':
            raw=[]
            print(cat)
            for page in range(1, 200):
                url = f'http://www.ironman.com/triathlon/events/americas/ironman/world-championship/results.aspx?p={page}&race=worldchampionship&rd={date}&agegroup={cat}&y={year}&ps=20#axzz5YeQSboHx'
                try: 
                    df = pd.read_html(url)
                except Exception as e:  
                    print(e)
                    print(f'total pages:{page-1}')
                    break
                raw.append(df[0])
                if df[0].shape[0] < 20:
                    print(f'total pages:{page}')
                    break
            data = pd.concat(raw)
            data['Agegroup'] = np.array([cat for index in range(data.shape[0])])
            print(f'done with {cat}')
        Raw.append(data)
    mega_data = pd.concat(Raw)
    return mega_data

def scraper_female(date, year): 
    raw=[]
    for page in range(1, 200): 
        url = f'http://www.ironman.com/triathlon/events/americas/ironman/world-championship/results.aspx?p={page}&race=worldchampionship&rd={date}&sex=F&y={year}&ps=20#axzz5YeQSboHx'
        df = pd.read_html(url)
        raw.append(df[0])
        if df[0].shape[0] < 20: 
            break
    data = pd.concat(raw)
    return data
def get_gender(array_no_gen, array_female): 
    f_names = list(array_female.Name)
    all_names = list(array_no_gen.Name)
    gender = []
    for name in all_names: 
        if name in f_names: 
            gender.append(1)
        else: 
            gender.append(2)
    print(f'no of females in gender list : {gender.count(1)}')
    print(f'no of females in in female database : {len(f_names)}')
    print(f'gender rows: {len(gender)} is equal to all_names rows : {len(all_names)}?')
    print(f'{len(gender) == len(all_names)}')
    return gender

def year_scraper_allinfo(date, year):
    #get df for all athletes that inlcuds agegroup but no gender for year A (e.g)
    df_no_gender = scraper_category(date, year)
    #get df for only females for year A
    df_female = scraper_female(date, year)
    #create an array that iterates over df_no_gender.Name to find same names in..
    # df_female.Name and return 1 if matches or 2 if not. 
    gender = get_gender(df_no_gender, df_female)
    print(len(gender))
    #add gender column to df_no_gender
    df_no_gender['Gender'] = np.array(gender)
    return df_no_gender

def scrape_all_years(date_list): 
    df_all_years = {}
    for date in date_list: 
        year = int(str(date)[:4])
        df_year = year_scraper_allinfo(date, year)
        df_all_years[str(year)]=df_year
        print(f'Done with {str(year)}')
    return df_all_years

In [315]:
#create a dictionary of dataframes for each year
dates_2008_2018 = dates[:11]
df_list_years = scrape_all_years(dates_2008_2018)

Pro
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with Pro
18-24
done with page:1
done with page:2
done with page:3
done with page:4
done with 18-24
25-29
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with 25-29
30-34
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with page:10
done with page:11
done with page:12
done with page:13
done with page:14
done with 30-34
35-39
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with page:10
done with page:11
done with page:12
done with page:13
done with page:14
done with page:15
done with page:16
done with 35-39
40-44
done with page:1
done with page:2
done with page:3
done with

done with page:7
done with page:8
done with page:9
done with page:10
done with page:11
done with page:12
done with page:13
No tables found
done with 30-34
35-39
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with page:10
done with page:11
done with page:12
done with page:13
done with page:14
done with page:15
done with page:16
done with page:17
done with 35-39
40-44
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with page:10
done with page:11
done with page:12
done with page:13
done with page:14
done with page:15
done with page:16
done with page:17
done with page:18
done with page:19
done with 40-44
45-49
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with pa

done with page:13
done with page:14
done with page:15
done with page:16
done with 45-49
50-54
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with page:10
done with page:11
done with 50-54
55-59
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with 55-59
60-64
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with 60-64
65-69
done with page:1
done with page:2
done with page:3
done with 65-69
no of females in gender list : <built-in method count of list object at 0x1250e5508>
no of females in in female database : 572
gender rows: 1981 is equal to all_names rows : 1981?
True
1981
Done with 2012
Pro
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with Pro
18-24
done with page:1
done with page:2
done with page:3
done with page:

done with page:16
done with page:17
done with 35-39
40-44
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with page:10
done with page:11
done with page:12
done with page:13
done with page:14
done with page:15
done with page:16
done with 40-44
45-49
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with page:8
done with page:9
done with page:10
done with page:11
done with page:12
done with 45-49
50-54
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with page:6
done with page:7
done with 50-54
55-59
done with page:1
done with page:2
done with page:3
done with page:4
done with page:5
done with 55-59
60-64
done with page:1
done with page:2
done with page:3
No tables found
done with 60-64
65-69
done with page:1
done with page:2
done with 65-69
no of females in gender l

In [ ]:
# df_list_years2 = scrape_all_years(dates_2008_2018)

In [325]:
#create csv files
for year in list(range(2008,2019)): 
    df_list_years[str(year)].to_csv(f'data/Runningman{year}')